# Omejen krožni problem treh teles

V nalogi rešujemo problem premikanja sonde Appolo, ki je poslana iz Zemljine orbite na tir z vrnitvijo brez potiska, tako da obkroži Luno in se vrne nazaj v Zemljino orbito.

Rešujemo sistem diferencialnih enačb v koordinatnem sistemu, kjer Zemlja in Luna mirujeta.

Ozančimo z $M$ maso Zemlje in z $m$ maso Lune, enačbe gibanja zapišemo v vrtečem koordinatnem sistemu, kjer masi $M$ in $m$ mirujeta. Torej:

$$
\mu = \frac{m}{M+m} \quad \text{in} \quad \bar{\mu} = \frac{M}{M+m}
$$

Masa $M$ je v točki $(-\mu, 0, 0)$, masa $m$ pa v točki $(1-\mu, 0, 0)$. Oddaljenost sonde pa potem zapišemo z:

$$
R = R(x,y,z) = \sqrt{(x + \mu)^2 + y^2 + z^2} \\
$$
$$
r = r(x,y,z) = \sqrt{(x - 1 + \mu)^2 + y^2 + z^2} \\
$$

Sistem diferencialnih enačb gibanja sonde je:
$$
\begin{align*}
\ddot{x} &= x + 2\dot{y} - \frac{\bar{\mu}}{R^3}(x+\mu) - \frac{\mu}{r^3}(x-\bar{\mu}) \\
\ddot{y} &= y - 2\dot{x} - \frac{\bar{\mu}}{R^3} y - \frac{\mu}{r^3} y \\
\ddot{z} &= -\frac{\bar{\mu}}{R^3} z - \frac{\mu}{r^3} z \\
\end{align*}
$$

Najprej sistem enačb drugega reda prevedemo na sistem prvega reda.

$$
\begin{align*}
\dot{x} &= v_x \quad \dot{y} = v_y \quad \dot{z} = v_z \\
x &= v_x t + x_0 \quad y = v_y t + y_0 \quad z = v_z t + z_0 \\
\dot{v_x} &= x + 2v_y - \frac{\bar{\mu}}{R^3}(x+\mu) - \frac{\mu}{r^3}(x-\bar{\mu}) \\
\dot{v_y} &= x + 2v_y - \frac{\bar{\mu}}{R^3}(x+\mu) - \frac{\mu}{r^3}(x-\bar{\mu}) \\
\dot{v_z} &= y - 2v_x - \frac{\bar{\mu}}{R^3} y - \frac{\mu}{r^3} y \\
\end{align*}
$$

In [12]:
# Dodajanje implementacije v Jupyter notebook
include("../src/main.jl");

In [9]:
# Almost stable 2.75 0.1
EarthRadius=6357.5*10^3
EarthMass=5.972 * 10^24
lowearthorbit = 185 * 10^3 + EarthRadius

earthmoon=384400*10^3
M_radius= 1737.4*10^3
M_mass = 7.34767309 * 10^22

injection_angle = 180.0 * pi/180
inj_alt = (185 * 10^3 + lowearthorbit) / (EarthRadius + M_radius + earthmoon)
pos0 = inj_alt * Vector([cos(injection_angle), sin(injection_angle), 0.0]) - Vector([mu, 0.0, 0.0])
speed0 = 15.0
v0 = speed0 * Vector([cos(injection_angle - pi/2.0), sin(injection_angle - pi/2.0), 0.0])

# Stable orbit 6.0 -0.04
x0 = Vector([
    pos0[1], pos0[2], pos0[3],
    v0[1], v0[2], v0[3]]);

In [13]:
mass_earth = 5.974e24;
mass_moon = 7.348e22;
mu = mass_moon / (mass_earth + mass_moon);

init_guess = [-mu-0.1, 0.0, 0.0, 0.0, 30.0, 0.0]
best_cond = find_moon_flyby(init_guess, 1000, 10000, 1e-6)
println("Best condition: ", best_cond)

yn, tn = rocket_rk4(dF, best_cond, 0.0001, 100)
ys = yn[:,2];
xs = yn[:,1];
println("Final position: ", ys[:, end])
x_m = (1.0 - mu) * cos.(LinRange(0, 2*pi, 100))
y_m = (1.0 - mu) * sin.(LinRange(0, 2*pi, 100))
xlims = (minimum(xs)-0.1, maximum(xs)+0.1)
scatter([-mu], [0.], label="Earth", color="green", ms=2)
scatter!([1.0-mu], [0.], label="Moon", color="red", ms=1)
plot!(x_m, y_m, label="Moon orbit", color="blue", lw=0.5)
plot!(xs, ys, label="x-y", xlabel="x", ylabel="y", title="Raketa", lw=.7, color="black", aspect_ratio=1.0, size=(800, 800))  
